In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch import multiprocessing
from torch import nn
from tensordict.nn import TensorDictModule

from collections import defaultdict

In [2]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch import multiprocessing

In [3]:
is_fork = multiprocessing.get_start_method() == "fork"

device = (
    torch.device(0)
    if torch.cuda.is_available() and not is_fork
    else torch.device("cpu")
)

## Environment Preparation

#### Load unity environment using `mlagents_envs`

In [4]:
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.environment import UnityEnvironment

channel = EngineConfigurationChannel()
env_path = "C:/Users/Pawel/Documents/Unity_Project/warehouse-bot-training/environment_builds/test_env_simplified/Warehouse_Bot.exe"

from torchrl.envs import UnityMLAgentsEnv

unity_env = UnityEnvironment(
  file_name=env_path,
  side_channels=[channel],
  additional_args=["-batchmode", "-nographics"]
)
channel.set_configuration_parameters(time_scale=10)

c:\Users\Pawel\anaconda3\envs\mlagents\lib\site-packages\mlagents_envs\environment.py:94: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  unity_communicator_version = StrictVersion(unity_com_ver)


#### Transform environment from `mlagents` to `gymnasium`

In [5]:
import gymnasium as gym

In [6]:
print(gym.__version__)

1.0.0


In [7]:
from env_gymnasium_wrapper import UnityGymWrapper

gymnasium_env = UnityGymWrapper(unity_env)

#### Creating stable_baselines3 model

In [8]:
# from stable_baselines3.common.env_util import make_vec_env
# from stable_baselines3.common.vec_env import SubprocVecEnv

# env = make_vec_env(gymnasium_env, n_envs=8, vec_env_cls=SubprocVecEnv)
# model = PPO("MlpPolicy", env, device="cpu")
# model.learn(total_timesteps=25_000)

In [9]:
from stable_baselines3 import PPO

# model = PPO("MlpPolicy", gymnasium_env, verbose=1,
#             learning_rate=3e-4,
#             n_steps=10240,
#             batch_size=256,
#             n_epochs=6,
#             clip_range=0.2,
#             gamma=0.99,
#             gae_lambda=0.96,
#             seed=0,
#             ent_coef=0.005,
#             vf_coef=0.5,
# )

model = PPO.load('./saved_models/find-warehouse_2', gymnasium_env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [10]:
model.learn(total_timesteps=750_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 75.1     |
|    ep_rew_mean     | 85       |
| time/              |          |
|    fps             | 495      |
|    iterations      | 1        |
|    time_elapsed    | 20       |
|    total_timesteps | 10240    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 81.5        |
|    ep_rew_mean          | 83.7        |
| time/                   |             |
|    fps                  | 490         |
|    iterations           | 2           |
|    time_elapsed         | 41          |
|    total_timesteps      | 20480       |
| train/                  |             |
|    approx_kl            | 0.005173945 |
|    clip_fraction        | 0.0595      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.326      |
|    explained_variance   | 0.973       |
|    learning_rate        | 0.

In [11]:
model.save('./saved_models/find-warehouse_3_long')